In [ ]:
import os
import pandas as pd
from tkinter import Tk, Label, Button, filedialog, RIGHT, Y, Scrollbar
from tkinter import ttk

os.makedirs("uploads", exist_ok=True)

# Function to analyze and categorize packets
def analyze_packets(filepath):
    try:
        
        df = pd.read_csv(filepath)

        # categories
        normal = []
        suspicious = []
        malicious = []

        for _, row in df.iterrows():
            if row['Protocol'] not in ['TCP', 'UDP', 'ICMP']:
                malicious.append(f"Packet #{row['No.']} - {row['Protocol']}: {row['Info']}")
            elif row['Length'] > 1500:
                suspicious.append(f"Packet #{row['No.']} - Length: {row['Length']}")
            elif 'malicious' in str(row['Info']).lower():  
                malicious.append(f"Packet #{row['No.']} - Info: {row['Info']}")
            else:
                normal.append(f"Packet #{row['No.']} - {row['Protocol']}: {row['Info']}")

        for row in treeview.get_children():
            treeview.delete(row)

        for item in normal:
            treeview.insert("", "end", values=("Normal", item))
        for item in suspicious:
            treeview.insert("", "end", values=("Suspicious", item))
        for item in malicious:
            treeview.insert("", "end", values=("Malicious", item))

    except Exception as e:
        for row in treeview.get_children():
            treeview.delete(row)
        treeview.insert("", "end", values=("Error", f"Error analyzing packets: {e}"))

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename(
        filetypes=[("CSV Files", "*.csv")],
        title="Select a Wireshark CSV File"
    )
    if file_path:
        filename = os.path.basename(file_path)
        dest_path = os.path.join("uploads", filename)
        os.rename(file_path, dest_path)
        file_label.config(text=f"Uploaded File: {filename}")
        analyze_packets(dest_path)

root = Tk()
root.title("Network IDS - Tkinter")
root.geometry("800x600")  # Set window size
root.config(bg="#f2f2f2")  # Set background color

file_label = Label(root, text="No file uploaded yet.", fg="blue", bg="#f2f2f2", font=("Arial", 12, "italic"))
file_label.pack(pady=20)

# Style for buttons
upload_btn = Button(root, text="Upload CSV File", command=upload_file, bg="#4CAF50", fg="white", font=("Arial", 12), relief="raised", padx=10, pady=10)
upload_btn.pack(pady=10)

# Treeview for structured table display
columns = ("Category", "Packet Details")
treeview = ttk.Treeview(root, columns=columns, show="headings", height=15)

treeview.heading("Category", text="Category", anchor="center")
treeview.heading("Packet Details", text="Packet Details", anchor="w")

treeview.column("Category", width=100, anchor="center")
treeview.column("Packet Details", width=500, anchor="w")

scrollbar = Scrollbar(root, orient="vertical", command=treeview.yview)
treeview.configure(yscrollcommand=scrollbar.set)
scrollbar.pack(side=RIGHT, fill=Y)

treeview.pack(padx=20, pady=20)

root.mainloop()
